# Reading Data

To do any data science, the first thing you're going to need is data. Let's start to look at a few common ways of reading data in Python. Along the way, we'll take a look at some language constructs that we'll use throughout the day and really start to get familiar with coding in Python.

### File handling in Python

First, let's look at file objects in Python. We get a file handle by using the **open** function.

In [20]:
ls

0 - Introduction.ipynb
1 - Reading Data.ipynb
2 - Data Wrangling with Pandas.ipynb
3 - Exploratory Analysis with Pandas.ipynb
4 - Exploratory Plotting.ipynb
5 - Modeling with scikit-learn.ipynb
Mem_new_stuff.ipynb
README.md
data/
environment.yaml
img/
imports-test.py
introduction.pdf
introduction.pptx
load_data.py
python-tutorial.ipynb
solutions/


In [12]:
csv_file = open("data/health_inspection_chi_sample.csv") #opens file handle

File objects are lazy **iterators** (here, *stream objects*). Lazy means that they only do things, in this case read data, when you ask them to. You can call **next** on iterator objects to explicitly get the next item.

In [27]:
!head -1 data/health_inspection_chi_sample.csv

address,aka_name,city,dba_name,facility_type,inspection_date,inspection_id,inspection_type,latitude,license_,location,longitude,results,risk,state,violations,zip


In [13]:
line = next(csv_file)

In [14]:
print(line)

address,aka_name,city,dba_name,facility_type,inspection_date,inspection_id,inspection_type,latitude,license_,location,longitude,results,risk,state,violations,zip



Here, the next item is a string which is the line in a file. In this case, it's the headers of the `csv` file.

The nice thing about **iterators** is that you can iterate through them with a for loop.

In [28]:
for line in csv_file:
    pass # I don't want to do anything just keep going

In [29]:
print(line) # this will be the last line of the file

 | 41. PREMISES MAINTAINED FREE OF LITTER, UNNECESSARY ARTICLES, CLEANING  EQUIPMENT PROPERLY STORED - Comments: CORRECTED. | 41. PREMISES MAINTAINED FREE OF LITTER, UNNECESSARY ARTICLES, CLEANING  EQUIPMENT PROPERLY STORED - Comments: OBSERVED STANDING WASTE WATER INSIDE OF THE MOP BUCKET IN THE REAR PREP AREA AND A WET MOP ON THE FLOOR; INSTRUCTED TO DISPOSE OF STANDING WASTE WATER WHEN DONE USING AND TO HANG WET MOPS UPWARD TO DRY.",60651.0



After consuming the file, we need to close the open file-handle.

In [30]:
csv_file.close()

We can avoid that by using what's called a **context manager** in Python. This is a really nice abstraction. You use context managers via the **with** statement.

In [31]:
with open("data/health_inspection_chi_sample.csv") as csv_file:
    for line in csv_file:
        pass

By using the `open` function as a context manager, we get an automatic call to close the open file when we exit the context (determined by the indentation level). When working with files non-interactively, you'll almost always want to use open as a context manager.

## Exercise

Write some code that iterates through the file `data/health_inspection_chi_sample.json` twice. Only call `open` once, however, then close the file. Can you find out, programatically, how many characters are in the file (hint: have a look at the `tell` method of the file object)?

In [ ]:
# Type your solution here

In [ ]:
# %load solutions/read_json_twice.py
json_file = open('data/health_inspection_chi_sample.json')

for line in json_file:
    pass

print(json_file.tell())

json_file.seek(0)

for line in json_file:
    pass


### Reading Data: CSVs

We peaked at this file above. We might already see some things we're going to have to clean up. Let's look at how we may do this. As always, Python has *batteries included*. The `csv` built-in module is great for basic data-munging needs.

In [33]:
import csv
from pprint import pprint

Let's open a csv file and use csv's `reader` function to read in the data.

In [34]:
csv_file = open("data/health_inspection_chi_sample.csv")

reader = csv.reader(csv_file)

`csv.reader` also returns a lazy iterator.

In [35]:
headers = next(reader)

In [36]:
pprint(headers)

['address',
 'aka_name',
 'city',
 'dba_name',
 'facility_type',
 'inspection_date',
 'inspection_id',
 'inspection_type',
 'latitude',
 'license_',
 'location',
 'longitude',
 'results',
 'risk',
 'state',
 'violations',
 'zip']


In [37]:
line = next(reader)

In [38]:
pprint(line)

['5255 W MADISON ST ',
 'RED SNAPPER FISH CHICKEN & PIZZA',
 'CHICAGO',
 'RED SNAPPER FISH CHICKEN & PIZZA',
 'Restaurant',
 '2016-09-26T00:00:00.000',
 '1965287',
 'Canvass',
 '41.880236543865834',
 '1991820.0',
 "{'type': 'Point', 'coordinates': [-87.757220392117, 41.880236543866]}",
 '-87.7572203921175',
 'Pass w/ Conditions',
 'Risk 1 (High)',
 'IL',
 '35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTED PER CODE: GOOD REPAIR, '
 'SURFACES CLEAN AND DUST-LESS CLEANING METHODS - Comments: MUST CLEAN THE '
 'WALLS AT WALL BASE NEAR THE MIXER IN REAR OF PREMISES AND THE PREP AREA OF '
 'FOOD SPILLS AND CLEAN THE WALL VENT IN PREP AREA ,INSTRUCTED TO CLEAN AND '
 'MAINTAIN AREA | 33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSILS CLEAN, FREE '
 'OF ABRASIVE DETERGENTS - Comments: MUST CLEAN THE INTERIOR PANEL OF THE ICE '
 'MACHINE IN REAR OF PREMISES | 34. FLOORS: CONSTRUCTED PER CODE, CLEANED, '
 'GOOD REPAIR, COVING INSTALLED, DUST-LESS CLEANING METHODS USED - Comments: '
 'MUST CLEAN 

The biggest difference in using `csv.reader` vs. iterating through the file is that it automatically splits the csv on commas and returns the line of the file split into a list.

You can control this behavior through a `Dialect` object. By default, `csv.reader` uses a Dialect object called "excel." Here let's look at the attributes of the excel dialect. Don't worry too much about the code used to achieve this. We'll look more at this later.

In [39]:
items = csv.excel.__dict__.items()

pprint({key: value for key, value in items if not key.startswith("_")})

{'delimiter': ',',
 'doublequote': True,
 'lineterminator': '\r\n',
 'quotechar': '"',
 'quoting': 0,
 'skipinitialspace': False}


You might also find `DictReader` to be useful. 

First, let's back up in the file to the beginning.

In [40]:
csv_file.seek(0)

0

In [41]:
reader = csv.DictReader(csv_file)

In [42]:
pprint(next(reader))

OrderedDict([('address', '5255 W MADISON ST '),
             ('aka_name', 'RED SNAPPER FISH CHICKEN & PIZZA'),
             ('city', 'CHICAGO'),
             ('dba_name', 'RED SNAPPER FISH CHICKEN & PIZZA'),
             ('facility_type', 'Restaurant'),
             ('inspection_date', '2016-09-26T00:00:00.000'),
             ('inspection_id', '1965287'),
             ('inspection_type', 'Canvass'),
             ('latitude', '41.880236543865834'),
             ('license_', '1991820.0'),
             ('location',
              "{'type': 'Point', 'coordinates': [-87.757220392117, "
              '41.880236543866]}'),
             ('longitude', '-87.7572203921175'),
             ('results', 'Pass w/ Conditions'),
             ('risk', 'Risk 1 (High)'),
             ('state', 'IL'),
             ('violations',
              '35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTED PER CODE: '
              'GOOD REPAIR, SURFACES CLEAN AND DUST-LESS CLEANING METHODS - '
              'Comments: 

In [43]:
csv_file.close()

One last handy trick with the `csv` module is to use `Sniffer` to determine the csv dialect for you.

In [ ]:
file_name = "data/health_inspection_chi_sample.csv"

with open(file_name) as csv_file:
    
    try:
        dialect = csv.Sniffer().sniff(csv_file.read(1024))
    except csv.Error as err:
        # log that this file format couldn't be deduced
        print(f"The format of {file_name} could not be detected.")
    else:
        csv_file.seek(0)
    
        dta = csv.reader(csv_file, dialect=dialect)

A big part of increased productivity is saving yourself some work later. The first rule of dealing with data is probably something like "The data is against you. Act accordingly."

We set this block of code up to be pretty paranoid. Software engineers call this **defensive programming**. It's a good habit to get into when you're doing any data science work in Python. 

There are three things to note here. First, is the use of `Sniffer` at all. If `file_name` is a standard csv format, we'll be able to read it.

The second is using a `try/except/else` block. In a `try/except` block, any exception that is raised will trigger the code in the except block. If no exception is raised, the *optional* else block will run.

Let's take a look at a toy example to fix ideas.

In [ ]:
try:
    1/0
except:
    print("Something went wrong.")

In [ ]:
try:
    1/0
except ZeroDivisionError as err:
    print("Something went wrong.")
    raise(err)

In [ ]:
try:
    1/0
except FileNotFoundError:
    print("This error isn't raised, so we're not here.")

The final thing to note in the block above is the use of `print` to provide some information about what went wrong. Logging is another really good habit to get into, and print statements are the dead simplest way to log the behavior of your code.

In practice, you probably don't want to use `print`. You want to use the [logging](https://docs.python.org/3/library/logging.html) module, but we're not going to talk about best practices in logging anymore today.

### Reading Data: json

Perhaps the second most common file format after CSVs is the `JSON` format. JSON stands for JavaScript Object Notation. When reading data from an API, for example, you will often encounter json files.

Each line in the file `data/health_inspection_chi_sample.json` is a single json object that represents the same data above. 

In [44]:
!head -n 1 data/health_inspection_chi_sample.json

{"address":"5255 W MADISON ST ","aka_name":"RED SNAPPER FISH CHICKEN & PIZZA","city":"CHICAGO","dba_name":"RED SNAPPER FISH CHICKEN & PIZZA","facility_type":"Restaurant","inspection_date":"2016-09-26T00:00:00.000","inspection_id":1965287,"inspection_type":"Canvass","latitude":41.8802365439,"license_":1991820.0,"location":{"type":"Point","coordinates":[-87.7572203921,41.8802365439]},"longitude":-87.7572203921,"results":"Pass w\/ Conditions","risk":"Risk 1 (High)","state":"IL","violations":"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTED PER CODE: GOOD REPAIR, SURFACES CLEAN AND DUST-LESS CLEANING METHODS - Comments: MUST CLEAN THE WALLS AT WALL BASE NEAR THE MIXER IN REAR OF PREMISES AND THE PREP AREA OF FOOD SPILLS AND CLEAN THE WALL VENT IN PREP AREA ,INSTRUCTED TO CLEAN AND MAINTAIN AREA | 33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSILS CLEAN, FREE OF ABRASIVE DETERGENTS - Comments: MUST CLEAN THE INTERIOR PANEL OF THE ICE MACHINE IN REAR OF PREMISES | 34. FLOORS: CONSTRUCTED PE

We can use the `json` module to read and manipulate json data.

In [45]:
import json

Since each line is a json object here, we need to iterate over the file and parse each line. We use the `json.loads` function here for "load string." The similar function `json.load` takes a file-like object.

In [46]:
dta = []

with open("data/health_inspection_chi_sample.json") as json_file:
    for line in json_file:
        line = json.loads(line)
        dta.append(line)

pprint(dta[0])

{'address': '5255 W MADISON ST ',
 'aka_name': 'RED SNAPPER FISH CHICKEN & PIZZA',
 'city': 'CHICAGO',
 'dba_name': 'RED SNAPPER FISH CHICKEN & PIZZA',
 'facility_type': 'Restaurant',
 'inspection_date': '2016-09-26T00:00:00.000',
 'inspection_id': 1965287,
 'inspection_type': 'Canvass',
 'latitude': 41.8802365439,
 'license_': 1991820.0,
 'location': {'coordinates': [-87.7572203921, 41.8802365439], 'type': 'Point'},
 'longitude': -87.7572203921,
 'results': 'Pass w/ Conditions',
 'risk': 'Risk 1 (High)',
 'state': 'IL',
 'violations': '35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTED PER CODE: '
               'GOOD REPAIR, SURFACES CLEAN AND DUST-LESS CLEANING METHODS - '
               'Comments: MUST CLEAN THE WALLS AT WALL BASE NEAR THE MIXER IN '
               'REAR OF PREMISES AND THE PREP AREA OF FOOD SPILLS AND CLEAN '
               'THE WALL VENT IN PREP AREA ,INSTRUCTED TO CLEAN AND MAINTAIN '
               'AREA | 33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSILS CLEAN

`json.loads` places each json object into a Python dictionary, helpfully filling in `None` for `null` for missing values and otherwise preserving types. It also, works recursively as we see in the `location` field.

## Aside: List Comprehensions

Let's take a look at another Pythonic concept, introduced a bit above, called a **list comprehension**. This is what's called *syntactic sugar*. It's a concise way to create a list.

In [51]:
list1 = [i for i in range(1, 6)]
list1

[1, 2, 3, 4, 5]

Alternatively, we could have made this list by writing

In [49]:
result_list = []

for i in range(1, 6):
    result_list.append(i)

result_list

[1, 2, 3, 4, 5]

List comprehensions can contain logic.

In [52]:
x = ['a', 'b', 'c', 'd', '_e', '_f']

In [53]:
[i for i in x if not i.startswith('_')]

['a', 'b', 'c', 'd']

You can also use a an else clause. Notice the slightly different syntax.

In [54]:
[i if not i.startswith('_') else 'skipped' for i in x]

['a', 'b', 'c', 'd', 'skipped', 'skipped']

List comprehensions can be nested, though it's usually best practices not to go overboard. They can quickly become difficult to read.

In [ ]:
matrix = [
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
]

Notice that we write this from outside to in.

In [ ]:
[element for row in matrix for element in row]

You can also create **dictionary comprehensions**.

In [ ]:
pairs = [
    ("first_initial", "J"), 
    ("last_name", "Doe"), 
    ("address", "1234 Main Street")
]

In [ ]:
{key: value for key, value in pairs}

## Exercise

Returning to the code that we introduced above, we can take further control over how a file with json objects is read in by using the `object_hook` argument. Say we wanted to remove the `location` field above. We don't need the `geoJSON` formatted information. We could do so with the `object_hook`. Write a function called `remove_entry` that removes the `'location'` field from each record in the `'data/health_inspection_chi_sample.json'` file.

Pass this function to the `object_hook` argument of `json.loads`. Be careful, `object_hook` will get called recursively on nested json objects.

In [ ]:
# Type your solution here

In [ ]:
%load solutions/object_hook_json.py

### Reading Data: pandas

Now let's take a look at using [pandas]() to read data.

#### Introducing Pandas

First, a few words of introduction for **pandas**. Pandas is a Python package providing fast, flexible, and expressive data structures designed to work with relational or labeled data. It is a high-level tool for doing practical, real world data analysis in Python.

You reach for pandas when you have:

* Tabular data with heterogeneously-typed columns
* Ordered and unordered (not necessarily fixed-frequency) time series data.
* Arbitrary matrix data with row and column labels

Almost any dataset can be converted to a pandas data structure for cleaning, transformation, and analysis.

Pandas is built on top of **numpy**, which we'll look a bit more at later.

First, let's import the pandas.

In [55]:
import pandas as pd
pd.set_option("max.rows", 6)

Pandas has facilities for reading csv files and files containing JSON records (and other formats). We can use `read_csv` for csv files.

In [58]:
df = pd.read_csv("data/health_inspection_chi_sample.csv")
df

,address,aka_name,city,dba_name,facility_type,inspection_date,inspection_id,inspection_type,latitude,license_,location,longitude,results,risk,state,violations,zip
0,5255 W MADISON ST,RED SNAPPER FISH CHICKEN & PIZZA,CHICAGO,RED SNAPPER FISH CHICKEN & PIZZA,Restaurant,2016-09-26T00:00:00.000,1965287,Canvass,41.880237,1991820.0,"{'type': 'Point', 'coordinates': [-87.75722039...",-87.757220,Pass w/ Conditions,Risk 1 (High),IL,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",60644.0
1,5958 W DIVERSEY AVE,TAQUERIA MORELOS,CHICAGO,TAQUERIA MORELOS,Restaurant,2014-02-06T00:00:00.000,1329698,Canvass,41.931250,2099479.0,"{'type': 'Point', 'coordinates': [-87.77590699...",-87.775907,Pass,Risk 1 (High),IL,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,60639.0
2,5400-5402 N CLARK ST,HAMBURGER MARY'S/MARY'S REC ROOM,CHICAGO,HAMBURGER MARY'S CHICAGO/MARY'S REC ROOM,Restaurant,2010-12-03T00:00:00.000,470787,SFP,41.979884,1933748.0,"{'type': 'Point', 'coordinates': [-87.66842948...",-87.668429,Fail,Risk 1 (High),IL,"6. HANDS WASHED AND CLEANED, GOOD HYGIENIC PRA...",60640.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,3251 E 92ND ST,VICTORY CENTRE OF SOUTH CHICAGO SLF,CHICAGO,VICTORY CENTRE OF SOUTH CHICAGO SLF,ASSISTED LIVING,2012-10-12T00:00:00.000,1296216,Canvass,41.728214,1968546.0,"{'type': 'Point', 'coordinates': [-87.54469279...",-87.544693,Pass,Risk 1 (High),IL,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",60617.0
998,4300 S UNION AVE,TUXEDO JUNCTION #2,CHICAGO,TUXEDO JUNCTION #2,Grocery Store,2011-07-14T00:00:00.000,585793,Canvass,41.816186,8974.0,"{'type': 'Point', 'coordinates': [-87.64345182...",-87.643452,Fail,Risk 2 (Medium),IL,"16. FOOD PROTECTED DURING STORAGE, PREPARATION...",60609.0
999,3820 W CHICAGO AVE,Bro-N-Laws,CHICAGO,Bro-N-Laws,Restaurant,2015-08-19T00:00:00.000,1566470,Complaint Re-Inspection,41.895513,2288968.0,"{'type': 'Point', 'coordinates': [-87.72216981...",-87.722170,Pass,Risk 1 (High),IL,45. FOOD HANDLER REQUIREMENTS MET - Comments: ...,60651.0


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
address            1000 non-null object
aka_name           991 non-null object
city               1000 non-null object
dba_name           1000 non-null object
facility_type      993 non-null object
inspection_date    1000 non-null object
inspection_id      1000 non-null int64
inspection_type    1000 non-null object
latitude           991 non-null float64
license_           998 non-null float64
location           991 non-null object
longitude          991 non-null float64
results            1000 non-null object
risk               1000 non-null object
state              1000 non-null object
violations         963 non-null object
zip                1000 non-null float64
dtypes: float64(4), int64(1), object(12)
memory usage: 132.9+ KB


`read_csv` is one of the best/worst functions in all of Python. It's great because it does just about everything. It's terrible, because it does just about everything. Chances are if you have a special case that pandas `read_csv` will accomodate your needs. Go ahead and have a look at the `docstring`.


```python
pd.read_csv?
```

In [61]:
pd.read_table?

`read_csv` returns a pandas DataFrame. We'll take a deeper dive into DataFrames next when we start to clean this data set.

The JSON counterpart to `read_csv` is `read_json`.

## Exercise

Use `pd.read_json` to read in the Chicago health inspections json sample in the `data` folder.

In [63]:
ls data

data/                              health_inspection_chi_sample.csv
data.zip                           health_inspection_chi_sample.json
get_data.py                        titanic.csv
health_inspection_chi.csv          zip_data.sh*


In [66]:
# Type your solution Here
df = pd.read_json("data/health_inspection_chi_sample.json",
                orient="records",
                lines=True,) #--because there are a many json objects not one
df

,address,aka_name,city,dba_name,facility_type,inspection_date,inspection_id,inspection_type,latitude,license_,location,longitude,results,risk,state,violations,zip
0,5255 W MADISON ST,RED SNAPPER FISH CHICKEN & PIZZA,CHICAGO,RED SNAPPER FISH CHICKEN & PIZZA,Restaurant,2016-09-26T00:00:00.000,1965287,Canvass,41.880237,1991820.0,"{'type': 'Point', 'coordinates': [-87.75722039...",-87.757220,Pass w/ Conditions,Risk 1 (High),IL,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",60644
1,5958 W DIVERSEY AVE,TAQUERIA MORELOS,CHICAGO,TAQUERIA MORELOS,Restaurant,2014-02-06T00:00:00.000,1329698,Canvass,41.931250,2099479.0,"{'type': 'Point', 'coordinates': [-87.77590699...",-87.775907,Pass,Risk 1 (High),IL,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,60639
2,5400-5402 N CLARK ST,HAMBURGER MARY'S/MARY'S REC ROOM,CHICAGO,HAMBURGER MARY'S CHICAGO/MARY'S REC ROOM,Restaurant,2010-12-03T00:00:00.000,470787,SFP,41.979884,1933748.0,"{'type': 'Point', 'coordinates': [-87.66842948...",-87.668429,Fail,Risk 1 (High),IL,"6. HANDS WASHED AND CLEANED, GOOD HYGIENIC PRA...",60640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,3251 E 92ND ST,VICTORY CENTRE OF SOUTH CHICAGO SLF,CHICAGO,VICTORY CENTRE OF SOUTH CHICAGO SLF,ASSISTED LIVING,2012-10-12T00:00:00.000,1296216,Canvass,41.728214,1968546.0,"{'type': 'Point', 'coordinates': [-87.54469279...",-87.544693,Pass,Risk 1 (High),IL,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",60617
998,4300 S UNION AVE,TUXEDO JUNCTION #2,CHICAGO,TUXEDO JUNCTION #2,Grocery Store,2011-07-14T00:00:00.000,585793,Canvass,41.816186,8974.0,"{'type': 'Point', 'coordinates': [-87.64345182...",-87.643452,Fail,Risk 2 (Medium),IL,"16. FOOD PROTECTED DURING STORAGE, PREPARATION...",60609
999,3820 W CHICAGO AVE,Bro-N-Laws,CHICAGO,Bro-N-Laws,Restaurant,2015-08-19T00:00:00.000,1566470,Complaint Re-Inspection,41.895513,2288968.0,"{'type': 'Point', 'coordinates': [-87.72216981...",-87.722170,Pass,Risk 1 (High),IL,45. FOOD HANDLER REQUIREMENTS MET - Comments: ...,60651


In [ ]:
# %load solutions/read_json.py
import pandas as pd

pd.read_json(
    "data/health_inspection_chi_sample.json",
    orient="records",
    lines=True,
)


### Reading Data: web data

So far, we've seen some ways that we can read data from disk. As Data Scientists, we often need to go out and grab data from the Internet.

Generally Python is "batteries included" and reading data from the Internet is no exception, but there are some *great* packages out there. [requests](http://docs.python-requests.org/en/master/) is one of them for making HTTP requests.

Let's look at how we can use the [Chicago Data Portal](https://data.cityofchicago.org/) API to get this data in the first place. (I originally used San Francisco for this, but the data was just too clean to be terribly interesting.)

In [67]:
import requests

We use the requests library to perform a GET request to the API, passing an optional query string via `params` to limit the returned number of records. The parameters are documented as part of the [Socrata Open Data API](https://dev.socrata.com/consumers/getting-started.html) (SODA).

In [68]:
#note the addition of parens to split the url into two lines
response = requests.get(
    "https://data.cityofchicago.org/'
    'resource/cwig-ma7x.json", 
    params="$limit=10"
)

Requests returns a [Reponse](http://docs.python-requests.org/en/master/api/#requests.Response) object with many helpful methods and attributes.

In [69]:
response

<Response [200]>

In [70]:
response.ok

True

In [71]:
dta = pd.read_json(response.content, orient='records')

We can use the `head` method to peak at the first 5 rows of data.

In [72]:
dta.head()

,:@computed_region_43wa_7qmu,:@computed_region_6mkv_f3dw,:@computed_region_awaf_s7ux,:@computed_region_bdys_3d7i,:@computed_region_vrxf_vc4k,address,aka_name,city,dba_name,facility_type,...,inspection_type,latitude,license_,location,longitude,results,risk,state,violations,zip
0,42,4447,33,264,73,3323 W 111TH ST,WONDERBURGER,CHICAGO,WONDERBURGER,Restaurant,...,License,41.691404,2626727,"{'type': 'Point', 'coordinates': [-87.70428241...",-87.704282,Fail,Risk 3 (Low),IL,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,60655.0
1,16,4451,46,536,76,1515 W DEVON AVE,LA UNICA FOOD MART,CHICAGO,"LA UNICA FOOD MART, INC.",RESTAURANT/GROCERY STORE,...,Complaint,41.997993,491,"{'type': 'Point', 'coordinates': [-87.66842788...",-87.668428,Fail,Risk 1 (High),IL,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,60660.0
2,46,14917,41,63,29,811 W FULTON MARKET ST,THE WING FULTON,NaN,THE WING FULTON,Restaurant,...,License,41.886684,2642303,"{'type': 'Point', 'coordinates': [-87.64800067...",-87.648001,Fail,Risk 2 (Medium),IL,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",NaN
3,41,21560,24,518,25,1610 W CHICAGO AVE,LOOP TAVERN,CHICAGO,LOOP TAVERN,Liquor,...,License,41.896175,2632196,"{'type': 'Point', 'coordinates': [-87.66777924...",-87.667779,Fail,Risk 3 (Low),IL,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,60622.0
4,39,22616,37,20,31,4521 N SHERIDAN RD,WABI SABI ROTARY,CHICAGO,WABI SABI ROTARY,Restaurant,...,License,41.964336,2647241,"{'type': 'Point', 'coordinates': [-87.65462704...",-87.654627,Pass w/ Conditions,Risk 1 (High),IL,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,60640.0


Of course, pandas can also load data directly from a URL, but I encourage you to reach for `requests` as often as you need it.

## Exercise

Try passing the URL above to `pd.read_json`. What happens?

In [81]:
!head -n 1 data/health_inspection_chi_sample.json #inefficent way to store data -column orientation


{"address":"5255 W MADISON ST ","aka_name":"RED SNAPPER FISH CHICKEN & PIZZA","city":"CHICAGO","dba_name":"RED SNAPPER FISH CHICKEN & PIZZA","facility_type":"Restaurant","inspection_date":"2016-09-26T00:00:00.000","inspection_id":1965287,"inspection_type":"Canvass","latitude":41.8802365439,"license_":1991820.0,"location":{"type":"Point","coordinates":[-87.7572203921,41.8802365439]},"longitude":-87.7572203921,"results":"Pass w\/ Conditions","risk":"Risk 1 (High)","state":"IL","violations":"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTED PER CODE: GOOD REPAIR, SURFACES CLEAN AND DUST-LESS CLEANING METHODS - Comments: MUST CLEAN THE WALLS AT WALL BASE NEAR THE MIXER IN REAR OF PREMISES AND THE PREP AREA OF FOOD SPILLS AND CLEAN THE WALL VENT IN PREP AREA ,INSTRUCTED TO CLEAN AND MAINTAIN AREA | 33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSILS CLEAN, FREE OF ABRASIVE DETERGENTS - Comments: MUST CLEAN THE INTERIOR PANEL OF THE ICE MACHINE IN REAR OF PREMISES | 34. FLOORS: CONSTRUCTED PE

In [74]:
# Type your solution here
url = ("https://data.cityofchicago.org/resource/cwig-ma7x.json")
df = pd.read_json(url, orient= 'records')

In [82]:
df.shape
df.head(1)
df = df.iloc[:,5:]

In [83]:
df.head()

,address,aka_name,city,dba_name,facility_type,inspection_date,inspection_id,inspection_type,latitude,license_,location,longitude,results,risk,state,violations,zip
0,4521 N SHERIDAN RD,WABI SABI ROTARY,CHICAGO,WABI SABI ROTARY,Restaurant,2019-03-21T00:00:00.000,2279748,Canvass Re-Inspection,41.964336,2492706,"{'type': 'Point', 'coordinates': [-87.65462704...",-87.654627,Pass,Risk 1 (High),IL,37. FOOD PROPERLY LABELED; ORIGINAL CONTAINER ...,60640.0
1,3323 W 111TH ST,WONDERBURGER,CHICAGO,WONDERBURGER,Restaurant,2019-03-21T00:00:00.000,2279749,License,41.691404,2626726,"{'type': 'Point', 'coordinates': [-87.70428241...",-87.704282,Fail,Risk 1 (High),IL,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",60655.0
2,811 W FULTON MARKET ST,THE WING FULTON,NaN,THE WING FULTON,Restaurant,2019-03-21T00:00:00.000,2279779,License,41.886684,2642303,"{'type': 'Point', 'coordinates': [-87.64800067...",-87.648001,Fail,Risk 2 (Medium),IL,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",NaN
3,3323 W 111TH ST,WONDERBURGER,CHICAGO,WONDERBURGER,Restaurant,2019-03-21T00:00:00.000,2279757,License,41.691404,2626727,"{'type': 'Point', 'coordinates': [-87.70428241...",-87.704282,Fail,Risk 3 (Low),IL,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,60655.0
4,1610 W CHICAGO AVE,LOOP TAVERN,CHICAGO,LOOP TAVERN,Liquor,2019-03-21T00:00:00.000,2279758,License,41.896175,2632196,"{'type': 'Point', 'coordinates': [-87.66777924...",-87.667779,Fail,Risk 3 (Low),IL,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,60622.0


In [ ]:
# %load solutions/read_url_json.py
import pandas as pd

url = ('https://data.cityofchicago.org/'
              'resource/cwig-ma7x.json?$limit=5')
pd.read_json(url, orient='records')


Notice how you can split a string across lines. This can be a very handy tip for improving readability, by splitting a string and putting it in parentheses, we preserve a single string.

In [84]:
("super "
 "long "
 "string "
 "split "
 "across "
 "lines")

'super long string split across lines'

#### Pandas DataReader

In addition to the core I/O functionality in pandas, there is also the [pandas-datareader](https://pandas-datareader.readthedocs.io/en/latest/) project. This package provides programmatic access to data sets from

* Yahoo! Finance (deprecated)
* Google Finance
* Enigma
* Quandl
* FRED
* Fama/French
* World Bank
* OECD
* Eurostat
* EDGAR Index (deprecated)
* TSP Fund Data
* Nasdaq Trader Symbol Definitions
* Morningstar
* Etc.

#### Further Resources

Sometimes we need to be resourceful in order to get data. Knowing how to scrape the web can really come in handy.

We're not going to go into details today, but you'll likely find libraries like [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/), [lxml](http://lxml.de/), and [mechanize](https://mechanize.readthedocs.io/en/latest/) to be helpful. 

There's also a `read_html` function in pandas that will quickly scrape HTML tables for you and put them into a DataFrame. 